In [1]:
## Carga de librerias
import cv2 #openCV open computr vision
import numpy as np # numeric python
from matplotlib import pyplot as plt
import math
import random
import time

In [2]:
def cargar(direccion):
    img = cv2.imread(direccion) #cargamos la imagen de la  direccion dada
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    return img

def show(imagen):
    plt.figure()
    plt.imshow(imagen)
    plt.axis('off')
    plt.show() #Mostramos el plot
    pass

# Implementacion de correccion de color para personas con daltonismo

### LMS Daltonization Algorithm
#### Su idea es utilizar la información perdida en la simulación de la ceguera al color y usar el espacio de color
#### LMS para compensar los colores que faltan en cada grupo / tipo de conos (largo (L), medio (M) y corto (S)) para
#### estar predecible para el espectador 

In [3]:
lmsColorSpaceM = np.array([[17.8824, 43.5161, 4.11935],
                           [3.45565, 27.1554, 3.86714],
                           [0.0299566, 0.184309, 1.46709]])


In [4]:
protanopiaM = np.array([[0, 2.02344, -2.52581],
                        [0, 1, 0],
                        [0, 0, 1]])
duteranopiaM = np.array([[1, 0, 0],
                         [0.49421, 0, 1.24827],
                         [0, 0, 1]])
tritanopiaM = np.array([[1, 0, 0],
                        [0, 1, 0],
                        [-0.395913, 0.801109, 0]])

In [5]:
rgbM = np.array([[0.089044447, -0.130504409, 0.116721066],
                 [0.113614708, -0.0102485335, 0.0540193266],
                 [-0.000365296,-0.004121614, 0.693511405]])

In [6]:
protanopiaSV = np.array([[0, 0, 0],
                         [0.7, 1, 0],
                         [0.7, 0, 1]])
duteranopiaSV = np.array([[1, 0.7, 0],
                         [0, 0, 0],
                         [0, 0.7, 1]])
tritanopiaSV = np.array([[1, 0, 0.7],
                         [0, 1, 0.7],
                         [0, 0, 0]])

In [22]:
def LMSDaltonization(image,t):
    img = np.copy(image)
    lms = np.copy(image)
    sx,sy = np.size(img,0),np.size(img,1)
    # Convert RGB image to LMS color space using equation (1)
    for n in range(0,sx):
        for m in range(0,sy):
            lms[n][m]= np.dot(lmsColorSpaceM,img[n][m]) # (1)
    #2 Simulate color-blindness using equation (2) for Protanopia, (3) for Duteranopia and (4) for Tritanopia
    lmsP = np.copy(image)
    lmsD = np.copy(image)
    lmsT = np.copy(image)
    for n in range(0,sx):
        for m in range(0,sy):
            if (t == 'p'):
                lmsP[n][m]= np.dot(protanopiaM,lms[n][m]) # (2)
            elif (t == 'd'):
                lmsD[n][m]= np.dot(duteranopiaM,lms[n][m]) # (3)
            elif (t == 't'):
                lmsT[n][m]= np.dot(tritanopiaM,lms[n][m]) # (4)
#     Convert LMS back to  RGB using equation (5), i= {P, D, T}
    rgb= np.copy(image)
    for n in range(0,sx):
        for m in range(0,sy):
            if (t == 'p'):
                rgb[n][m]= np.dot(rgbM,lmsP[n][m]) #(5)
            elif (t == 'd'):
                rgb[n][m]= np.dot(rgbM,lmsD[n][m]) #(5)
            elif (t == 't'):
                rgb[n][m]= np.dot(rgbM,lmsT[n][m]) #(5)
# Find Difference between original and simulated images by (6), (7) and (8)
    dif = np.copy(image)
    for n in range(0,sx):
        for m in range(0,sy):
            dif[n][m] = img[n][m]-rgb[n][m] # (6), (7) and (8)
# Shift colors towards visible spectrum by multiplying by error matrices using (9) for Protanopia, (10) for Duteranopia and (11) for Tritanopia
    mapP = np.copy(image)
    mapD = np.copy(image)
    mapT = np.copy(image)
    for n in range(0,sx):
        for m in range(0,sy):
            if (type == 'p'):
                mapP[n][m]= np.dot(protanopiaSV,dif[n][m])  # (9)
            elif (type == 'd'):
                mapD[n][m]= np.dot(duteranopiaSV,dif[n][m]) # (10)
            elif (type == 't'):
                mapT[n][m]= np.dot(tritanopiaSV,dif[n][m]) # (11)
# Add shifted colors to original image using (12), (13) and (14)
    rgbF = np.copy(image)
    for n in range(0,sx):
        for m in range(0,sy):
            if (type == 'p'):
                rgbF[n][m] =img[n][m]+mapP
                
            elif (type == 'd'):
                rgbF[n][m] = img[n][m]+mapD
            elif (type == 't'):
                rgbF[n][m] = img[n][m]+mapT
                print(rgbFT[n][m])
#     if (type == 'p'):
#         return  rgbFP
#     elif (type == 'd'):
#         return rgbFD
#     elif (type == 't'):
#         return rgbFT

SyntaxError: keyword can't be an expression (<ipython-input-22-63d119cdaa19>, line 55)

In [23]:
def test():
    test = cargar('Dataset/pdi3.PNG')
    
    btest = LMSDaltonization(test,'p')
    show(test)
#     show(btest)

    pass

test()

NameError: name 'rgbFP' is not defined

In [0]:
def imageToHSL(image):
    img = np.copy(image)
    sx,sy = np.size(img,0),np.size(img,1)
    for n in range(0,sx):
        for m in range(0,sy):
            h=l=s=0
            R,G,B = img[n][m]/255
            minimo = np.amin([R,G,B])
            maximo = np.amax([R,G,B])
            d = maximo - minimo
            #light
            l = (maximo+minimo)/2*100
            # excepcion
            if d == 0.0:
                h = 0
                s = 0
            #hue
            elif maximo == R:
                h = 60*((G-B))/d
            elif maximo == G:
                h = 60*((B-R)/d)+120
            elif maximo == B:
                h = 60*((R-G)/d)+240
            #saturate
            if d <0 :
                s = d/(1- (2*l-1))*100
            elif d >0 :
                s = d/(1- (2*l-1))*100
            img[n][m] = np.array([h,s,l])
    return img


In [0]:
def testColorBlind():
    test = cargar('Dataset/pdi1.PNG')
    b = imageToHSL(test)
    show(test)
    show(b)
    
testColorBlind()

In [0]:
def DominationColor(image):
    img = np.copy(image)
    sx,sy = np.size(img,0),np.size(img,1)
    for n in range(0,sx):
        for m in range(0,sy):
            h,s,l = img[n][m]
            if img[n][m] <= np.array([255,0,0]):
                h = 0.7*h
                s = 0.9*s
                l = 1.25*l
                img[n][m]=np.array([h,l,s])
            else:
                s = 1.1*s
                l = 0.9*l